## Climatic data



In [1]:
import ee
import geemap

# Authenticate to Earth Engine
try:
  ee.Initialize()
except Exception as e:
  ee.Authenticate()
  ee.Initialize()

from utils import export_image
from utils import map_image

# data_folder = 'projects/amazon-forest-regrowth/assets'
roi = ee.FeatureCollection("projects/amazon-forest-regrowth/assets/raw/biomes_br").geometry().dissolve()

months = ee.List.sequence(1, 12)
years = ee.List.sequence(1985, 2019)

ecoregions = (ee.FeatureCollection("RESOLVE/ECOREGIONS/2017").filterBounds(roi)
                .map(lambda feature: feature.intersection(roi)))

# Terraclim
https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE
Bring temperature and precipitation and calculate seasonality

In [3]:
terraclim = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE') \
              .filterDate('1985-01-01', '2019-12-31') \
              .map(lambda image : image.clip(roi)) \
              .map(lambda image: image.reduceResolution(ee.Reducer.median(), bestEffort=True, maxPixels=1024) \
                                       .reproject(crs='EPSG:4326', scale=10000))

# scaling to 0.1
maxtemp = terraclim.select('tmmx').map(lambda image: image.multiply(0.1))
mintemp = terraclim.select('tmmn').map(lambda image: image.multiply(0.1))
radiation = terraclim.select('srad').map(lambda image: image.multiply(0.1))
prec = terraclim.select('pr')

## Precipitation

For each regrowing forest pixel, we only consider rainfall AFTER the regrowth moment.
- Make a raster with mean yearly precipitation, temperature, and precipitation
- select only the values for years after regrowth. For years before regrowth, change the values to zero.

## Seasonality index

Walsh and Lawler 1981

In [ ]:
yearlist = range(1985, 2020) # Generate a list of years from 1985 to 2019

def seasonality_calc(year):
    # get mean monthly precipitation for the year
    yr_prec = prec.filter(ee.Filter.calendarRange(year, year, 'year'))
    mean_prec = yr_prec.reduce(ee.Reducer.mean())
    # Calculate absolute deviations of monthly precipitation from mean
    deviations = yr_prec.map(lambda month:month.subtract(mean_prec).abs())
    # Calculate sum of absolute deviations for each month
    sum_deviations = deviations.reduce(ee.Reducer.sum())
    # Calculate total annual precipitation
    total_annual_prec = yr_prec.reduce(ee.Reducer.sum())
    # Calculate Seasonality Index (SI)
    seasonality_index = sum_deviations.divide(total_annual_prec)
    return seasonality_index

tst = years.map(seasonality_calc)
yearly_SI = ee.ImageCollection.fromImages(tst)

# Convert ImageCollection to single Image
yearly_SI_image = yearly_SI.toBands()
new_band_names = ['si_{}'.format(year) for year in yearlist] # Append 'si_' to each year
yearly_SI_image = yearly_SI_image.rename(new_band_names)
# export_image(yearly_SI_image, "yearly_si")

In [ ]:
def meantemp_calc(year):
    # Filter both maxtemp and mintemp ImageCollections by year at once
    yearly_maxtemp = maxtemp.filter(ee.Filter.calendarRange(year, year, 'year')).reduce(ee.Reducer.mean())
    yearly_mintemp = mintemp.filter(ee.Filter.calendarRange(year, year, 'year')).reduce(ee.Reducer.mean())
    # Calculate the mean temperature directly
    mean_temp = yearly_maxtemp.add(yearly_mintemp).divide(2)
    return mean_temp

# Map the function over the years
mean_temp = ee.ImageCollection.fromImages(years.map(meantemp_calc)).toBands()

new_band_names = ['mean_temp_{}'.format(year) for year in yearlist] # Append 'si_' to each year
mean_temp = mean_temp.rename(new_band_names)

In [4]:
def mean_yearly(year, imgcol):
    mean_year = imgcol.filter(ee.Filter.calendarRange(year, year, 'year')).reduce(ee.Reducer.mean())
    return mean_year.set('year', year)

mean_prec = ee.ImageCollection.fromImages(years.map(prec)).toBands()

yearlist = range(1985, 2020) # Generate a list of years from 1985 to 2019
new_band_names = ['prec_{}'.format(year) for year in yearlist]
mean_prec = mean_prec.rename(new_band_names)

export_image(mean_prec, "mean_prec")

## Yearly CWD

Calculated as in Malhi et al 2009 and Aragão et al 2007, considering ET fixed as 100mm/month.

[equations](https://imgur.com/o4lVmM7)

In [3]:
"""
Calculate monthly cumulative water deficit
"""
def cwd_monthly(current_image, previous_dict):
    current_image = ee.Image(current_image)
    previous_dict = ee.Dictionary(previous_dict)
    previous_image = ee.Image(previous_dict.get('result_image'))
    result_image = previous_image.add(current_image).subtract(100)
    result_image = result_image.where(result_image.gt(0), 0) \
                                   .set('system:time_start', current_image.get('system:time_start'))
    return ee.Dictionary({
        'result_image': ee.Image(result_image).toFloat(),
        'cwd_list': ee.List(previous_dict.get('cwd_list')).add(result_image)})

# Use iterate to apply the function to each image
cwd_dict = ee.Dictionary(prec.iterate(cwd_monthly, ee.Dictionary({'result_image': ee.Image(0).toFloat(),
                                                               'cwd_list': ee.List([])})))
cwd_list = ee.List(cwd_dict.get('cwd_list'))
# cwd_list = cwd_list.map(lambda image: image.toFloat())

"""

"""
def mcwd_yearly(year):
    # Calculate the start and end indices for the current year
    start = ee.Number(year).subtract(1985).multiply(12)
    end = start.add(12)
    # Slice the list to get the images for the current year
    yr = ee.List(cwd_list).slice(start, end)
    cwd_a = ee.ImageCollection.fromImages(yr)
    mcwd_a = cwd_a.reduce(ee.Reducer.min())
    return mcwd_a.set('year', year)

# Map the function over the range
yearly_mcwd_img = ee.ImageCollection.fromImages(years.map(mcwd_yearly)).toBands()

yearlist = range(1985, 2020) # Generate a list of years from 1985 to 2019
new_band_names = ['mcwd_{}'.format(year) for year in yearlist] # Append 'si_' to each year
yearly_mcwd_img = yearly_mcwd_img.rename(new_band_names)
